## SetFit Few-Shot Classification

In questo notebook vengono mostrati i segmenti di codice e i procedimenti eseguiti per effettuare l'allenamento e l'inferenza del modello di classificazione.

L'obiettivo è quello di classificare le note di chiusura degli interventi dei tecnici in base alle aree di guasto possibili della macchina. Queste comprendono: CASSETTE, CT, NE, NF, NV e SHUTTER.
Il sistema deve poter categorizzare questi elementi testuali in maniera più o meno affidabile attraverso IA, includendo anche più di un'area per selezione.

La task che quindi dobbiamo eseguire si ricongiunge ad una classificazione di testo multi-label.

L'approccio utilizzato per la risoluzione consiste nel Few-Shot Learning, dove un modello IA di embedding viene allenato su un numero N di esempi reali per ogni singola label. Richiede più tempo e risorse hardware ma permette di ottenere risultati migliori in ambienti con pochi dati di allenamento e somprattutto categorizzazioni complesse come nel nostro caso.

Utilizziamo diverse librerie di HuggingFace come Sentence-Transformers e SetFit per la definizione e l'allenamento del modello IA, insieme a Datasets per la manipolazione dei dati necessari agli step di training, validation e test.



Qui di seguito nella prima cella definiamo la variabile d'ambiente per poter utilizzare la GPU durante l'allenamento.


In [124]:
import os

from sklearn.metrics import accuracy_score
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Come primo step andiamo ad importare i 3 dataset necessari a completare l'operazione di training.
Consistono nel dataset di training che presenta una varietà di dati labellizzati su cui il modello eseguirà l'allenamento, poi c'è il dataset di validation che contiene altri elementi labellizzati e conosciuti per validare l'accuratezza del modello in casi controllati.
Infine abbiamo il dataset di test, contenente una grande collezione di note di chiusura senza labellizzazioni revisionate in cui il modello verrà messo contro le selezioni dei tecnici.

I dataset sono estratti in pandas Dataframe da dei file Excel, e successivamente convertiti in Dataset filtrando le colonne ritenute non rilevanti per l'allenamento.

Le label vengono estratte dalle colonne del dataset di training nel formato "{label} ground-truth".

In [125]:
import pandas as pd
from datasets import Dataset

df_training = pd.read_excel("path-del-set-di-training")
df_validation = pd.read_excel("path-del-set-di-validation")
df_test = pd.read_excel("path-del-set-di-test")

dataset = Dataset.from_pandas(df_training.iloc[:, 9:16])
validation_dataset = Dataset.from_pandas(df_validation.iloc[:, 9:16])
test_dataset = Dataset.from_pandas(df_test.iloc[:, [0,1,2,3,4,5, 32, 49]])

features = dataset.column_names
features.remove("Closing Note")
features

['CASSETTE ground-truth',
 'CT ground-truth',
 'NE ground-truth',
 'NF ground-truth',
 'NV ground-truth',
 'SHUTTER ground-truth']

Eseguiamo ulteriori elaborazioni sui nostri dataset correnti, in cui rimappiamo le colonne delle singole label in una singola colonna "labels" contenente un vettore delle singole selezioni in formato binario.

Successivamente rimuoviamo tutte le righe in cui le note di chiusura risultano vuote.

In [126]:
from datasets import Dataset

dataset = dataset.map(lambda entry: {"labels": [entry[label] for label in features]})
validation_dataset = validation_dataset.map(lambda entry: {"labels": [entry[label] for label in features]})
validation_dataset = validation_dataset.map(lambda entry: {"text": entry["Closing Note"]})
test_dataset = test_dataset.map(lambda entry: {"labels": [entry[label] for label in features]})
test_dataset = test_dataset.map(lambda entry: {"text": entry["Closing Note"]})

C:\Users\fbrossi\PyCharmMiscProject\.venv1\Lib\site-packages\datasets\utils\_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
  obj.co_lnotab,  # for < python 3.10 [not counted in args]
Map: 100%|██████████| 4453/4453 [00:00<00:00, 4466.50 examples/s]


In [127]:
dictionary = {"text": dataset["Closing Note"], "labels": dataset['labels']}
train_dataset = Dataset.from_dict(dictionary)

train_dataset = Dataset.from_pandas(train_dataset.to_pandas().dropna())
train_dataset = Dataset.from_pandas(train_dataset.to_pandas().replace(r'^\s*$', "Empty", regex=True))

dictionary = {"text": validation_dataset["Closing Note"], "labels": validation_dataset['labels']}
validation_dataset = Dataset.from_dict(dictionary)

validation_dataset = Dataset.from_pandas(validation_dataset.to_pandas().dropna())
validation_dataset = Dataset.from_pandas(validation_dataset.to_pandas().replace(r'^\s*$', "Empty", regex=True))

test_dataset = Dataset.from_pandas(test_dataset.to_pandas().dropna())
test_dataset = Dataset.from_pandas(test_dataset.to_pandas().replace(r'^\s*$', "Empty", regex=True))

get_templated_dataset è un metodo di SetFit che permette di generare dati sintetici a seconda delle necessità.
Date le label e impostando il parametro multi_label = True, possiamo decidere un numero di esempi sintetici che verranno aggiunti per label seguendo un template uguale per tutte. Risulta essere molto semplice e modificabile, ma aiuta comunque per la classificazione.

In [128]:
from setfit import get_templated_dataset

train_dataset = get_templated_dataset(train_dataset, candidate_labels=features, sample_size=5, label_column="labels", multi_label=True, template="Il problema è del {}")

Si definisce una funzione per l'inizializzazione del modello che verrà usato dal trainer.
Vi è anche la possibilità di passare direttamente la variabile del modello, ma la funzione è vantaggiosa per motivi di versatilità, e la possibilità di implementare alcune funzionalità aggiuntive che la richiedono.

Qui definiamo delle parametrizzazioni di base tra cui:
    - la temperatura (rappresenta la creatività del modello ed è impostata a 0);
    - il numero, e la lista di label con cui deve rispondere;
    - la strategia di selezione della label (nel nostro caso "multi-output", ma anche "one-vs-rest"...).

In [130]:
import torch
from setfit import SetFitModel
import os
os.environ["WANDB_DISABLED"] = "true"

def model_init(params):
    params = {"device": torch.device("cuda"), 'out_features': 6, 'temperature': 0}
    return SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5",
                                       multi_target_strategy="multi-output", params=params, labels=features)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Successivamente specifichiamo nel dettaglio i parametri a cui il nostro Trainer sarà sottoposto, questo significa che ogni elemento che si definisce qui riguarderà la fase di allenamento e non sarà quindi necessario ridefinirle in qualsiasi altro utlizzo del modello post-allenamento.

Qui è anche possibile specificare dei parametri per il debugging.

Gli elementi che più interessano in questo caso sono: <br>
    <p>- <b>body_learning_rate</b> (definisce "quanto" il modello dovrà imparare ad ogni step dell'allenamento. Se troppo alto rischia di causare più facilmente "over-training");</p>
    <p>- <b>num_epochs</b> (rappresenta il numero di volte che il dataset viene attraversato nella sua interezza);</p>
    <p>- <b>batch_size</b> (il numero di sample processato per ogni step, un "chunk");</p>
    <p>- <b>warmup_proportion</b> (influisce sul learning_rate nei primi step di allenamento, mantenendo un basso valore prima di passare al learning_rate definito. Dovrebbe aiutare ad aumentare l'attenzione del modello);</p>
    <p>- <b>sampling_strategy</b> (riguarda il bilanciamento del numero di comparazioni per label. "unique" in questo caso non bilancia il peso delle label, garantendo comunque che vengano effettuate tutte le comparazioni senza duplicazioni).</p>

In [131]:
from setfit import TrainingArguments

args = TrainingArguments(
    # Parametri di training opzionali:
    body_learning_rate=1.9859376752033417e-05,
    num_epochs=2,
    batch_size=6,
    warmup_proportion=0.2,
    sampling_strategy="unique",
    # Parametri di debugging:
    logging_strategy="steps",
    logging_steps=1000,
    eval_strategy="steps",
    logging_first_step=True,
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    run_name="finetune-setfit",
    load_best_model_at_end=True
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Si inizializza poi il Trainer specificando i dataset di training e validation, gli argomenti e la funzione di inizializzazione del modello precedentemente definiti, cominciando lo step di Fine-Tuning.

In [133]:
from setfit import Trainer

trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    column_mapping={"text": "text", "labels": "label"},
)

trainer.train()

Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
C:\Users\fbrossi\PyCharmMiscProject\.venv1\Lib\site-packages\datasets\utils\_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
  obj.co_lnotab,  # for < python 3.10 [not counted in args]
Map: 100%|██████████| 148/148 [00:00<00:00, 10686.12 examples/s]


Si verifica l'accuratezza generale del modello allenato con una semplice metrica, contro il set di validation.

In [137]:
metrics = trainer.evaluate()
print(metrics)

Sono definite alcune funzioni utili per la manipolazione dei risultati.

In [140]:
import numpy
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import multilabel_confusion_matrix

def export_as_excel(filename, preds, bool):
    preds = map(lambda i: i.numpy().astype(numpy.int64).tolist(), preds)
    if bool:
        df_out = Dataset.to_pandas(test_dataset)
    else:
        df_out = Dataset.to_pandas(validation_dataset)

    print(preds)
    print(k.numpy().astype(numpy.int64).tolist() for k in preds)

    cassette, ct, ne, nf, nv, shutter = [], [], [], [], [], []


    for k in preds:
        cassette.append(int(k[0]))
        ct.append(int(k[1]))
        ne.append(int(k[2]))
        nf.append(int(k[3]))
        nv.append(int(k[4]))
        shutter.append(int(k[5]))

    df_out = pd.concat([df_out, pd.DataFrame({
                "CASSETTE Model": cassette,
                "CT Model": ct,
                "NE Model": ne,
                "NF Model": nf,
                "NV Model": nv,
                "SHUTTER Model": shutter
    })], axis=1)

    df_out = pd.concat([df_out, pd.DataFrame({
        "model_body": model.model_body
    })], axis=0)

    df_out.to_excel(filename, index=False)


def elaborate_pred(i):
    result = []
    count = 0
    for k in i:
        if k == 1:
            result.append(str(features[count]))
        count += 1
    return result

def confusion_matrix(w_dataset):
    res = multilabel_confusion_matrix(w_dataset['labels'], preds).ravel().tolist()
    n = 4
    res = [res[i:i + n] for i in range(0, len(res), n)]
    res = {"CASSETTE": res[0], "CT": res[1], "NE": res[2], "NF": res[3], "NV": res[4], "SHUTTER": res[5]}
    return res

def percentage(array):
    if not len(array) == 0:
        return round((sum(array) / len(array)) * 100, 2)
    else:
        return 0

def print_results(w_dataset):
    CASSETTE, CT, NF, NE, NV, SHUTTER, CRM = [], [], [], [], [], [], []
    ac_cassette, ac_ct, ac_nf, ac_ne, ac_nv, ac_shutter = [], [], [], [], [], []
    ac_cassette_pred, ac_ct_pred, ac_nf_pred, ac_ne_pred, ac_nv_pred, ac_shutter_pred = [], [], [], [], [], []

    array = []
    count = 0
    for i in preds:
        i = i.numpy().astype(numpy.int64).tolist()
        matching = w_dataset['labels'][count]
        print(str(w_dataset['text'][count]) + '\n' + str(elaborate_pred(matching)) + '\n' + str(elaborate_pred(i)) + '\n' + str(pred_proba[count]) + '\n')
        array.append(w_dataset['labels'][count] == i)

        ac_cassette.append(matching[0])
        ac_cassette_pred.append(i[0])
        ac_ct.append(matching[1])
        ac_ct_pred.append(i[1])
        ac_ne.append(matching[2])
        ac_ne_pred.append(i[2])
        ac_nf.append(matching[3])
        ac_nf_pred.append(i[3])
        ac_nv.append(matching[4])
        ac_nv_pred.append(i[4])
        ac_shutter.append(matching[5])
        ac_shutter_pred.append(i[5])

        count2 = 0
        for k in matching:
            truth_selection = k == 1
            pred_selection = i[count2] == 1
            if truth_selection or pred_selection:
                match count2:
                    case 0:
                        CASSETTE.append(k == i[count2])
                    case 1:
                        CT.append(k == i[count2])
                    case 2:
                        NE.append(k == i[count2])
                    case 3:
                        NF.append(k == i[count2])
                    case 4:
                        NV.append(k == i[count2])
                    case 5:
                        SHUTTER.append(k == i[count2])
            count2 += 1
        count += 1

    result = "Total: " + str(round(accuracy_score(w_dataset["labels"], preds), 2)) + " - " + str(len(array)) + "\nCASSETTE: " + str(round(balanced_accuracy_score(ac_cassette, ac_cassette_pred), 2)) + " - " + str(len(CASSETTE)) + "\nCT: " + str(round(balanced_accuracy_score(ac_ct, ac_ct_pred), 2)) + " - " + str(len(CT)) + "\nNE: " + str(round(balanced_accuracy_score(ac_ne, ac_ne_pred), 2)) + " - " + str(len(NE)) + "\nNF: " + str(round(balanced_accuracy_score(ac_nf, ac_nf_pred), 2)) + " - " + str(len(NF)) + "\nNV: " + str(round(balanced_accuracy_score(ac_nv, ac_nv_pred),2)) + " - " + str(len(NV)) + "\nSHUTTER: " + str(round(balanced_accuracy_score(ac_shutter, ac_shutter_pred),2)) + " - " + str(len(SHUTTER)) + '\n\n'
    print(str(result))

    print("TN, FP, FN, TP \n")
    print(str(confusion_matrix(w_dataset)) + '\n')

A seguito del completamento del Fine-Tuning, si salva il modello su disco in modo tale da poter farne riuso.

In [ ]:
model = trainer.model
model.save_pretrained("path-del-modello")

Quando si vuole utilizzare un modello esterno già presente su disco ad esempio, basta richiamare l'apposito metodo fornito da SetFitModel con il path e nome del modello.

Per eseguire una predizione basta richiamare il metodo predict dal model fornendo la lista di stringhe che si devono testare. In questo caso utilizziamo la colonna "text" del dataset di testing che contiene tutte le note dei tecnici.

Si può anche utilizzare il metodo predict_proba che fornisce le percentuali di confidenza sulle scelte che il modello ha preso per le predizioni.

In [ ]:
model = SetFitModel.from_pretrained("path-del-modello")

preds = model.predict(test_dataset['text'])
pred_proba = model.predict_proba(test_dataset['text'])

Si va a scrivere le predizioni all'interno di un nuovo file Excel per una più facile consultazione e analisi.

In [ ]:
export_as_excel('ClosingNotesResults.xlsx', preds, False)

Infine si va a stampare in output ogni singola predizione e le sue probabilità, accodando ai risultati svariati valori di accuratezza tra cui la balanced accuracy di ogni label e la subset accuracy del totale.

Si forniscono anche le informazioni di training relative al modello allenato in questo caso, per poi fornire un completo classification report tramite sklearn, approfondendo nel dettaglio molte altre metriche di valutazione che potrebbero risultare rilevanti per ulteriori procedure di tuning.

In [ ]:
print_results(validation_dataset)
print("body_learning_rate: " + str(args.body_learning_rate))
print("num_epochs: " + str(args.num_epochs))
print("batch_size: " + str(args.batch_size))
print("warmup_proportion: " + str(args.warmup_proportion))
print(classification_report(test_dataset['labels'], preds, target_names=features, zero_division=0))